# Data Collection and Processing

**Objective:**
This notebook gathers and merges data from three sources to create a comprehensive dataset for analysis:
1. **Olympic Medal Data (CSV):** Historical sports performance.
2. **UN GDP Data (CSV):** Economic wealth (GDP per Capita).
3. **World Bank Data (API):** Demographic data (Total Population and Active Population ages 15-64).

**Methodology:**
We merge datasets using the `merge_asof` (nearest year) strategy to handle time series discrepancies.

In [9]:
import pandas as pd
import numpy as np
from pandas_datareader import wb # API Library
import os
import sys

# Ensure pandas_datareader is installed
# !{sys.executable} -m pip install pandas-datareader

# Define File Paths
# Assumes structure: project/notebooks/this_file.ipynb and project/data/raw/files.csv
OLYMPICS_PATH = '../data/raw/Olympics (1896-2024).csv'
GDP_PATH = '../data/raw/SYB67_230_202411_GDP and GDP Per Capita.csv'

print("✅ Libraries loaded and paths defined.")

✅ Libraries loaded and paths defined.


In [10]:
print("1. Fetching Demographic Data from World Bank API...")

# Kodları tanımla
indicator_codes = ['SP.POP.TOTL', 'SP.POP.1564.TO']

try:
    # 1. Veriyi İndir
    df_api = wb.download(indicator=indicator_codes, country='all', start=1990, end=2023)
    df_api = df_api.reset_index()
    
    # 2. Sütun İsimlerini Garanti Olarak Değiştir
    # API bazen isimleri değiştirmez, biz elle zorluyoruz.
    # Önce sütun adlarını temizleyelim (boşluk vs varsa)
    df_api.columns = df_api.columns.str.strip()
    
    # Manuel Sözlük
    rename_map = {
        'SP.POP.TOTL': 'Total_Population',
        'SP.POP.1564.TO': 'Active_Population_Total',
        'country': 'Country',
        'year': 'Year'
    }
    df_api.rename(columns=rename_map, inplace=True)
    
    # Yıl sütununu sayıya çevir
    df_api['Year'] = df_api['Year'].astype(int)
    
    print(f"✅ API Success! Retrieved {len(df_api)} rows.")
    print("Sütunlar:", df_api.columns.tolist()) # Kontrol için sütunları yazdıralım
    display(df_api.head())
    
except Exception as e:
    print(f"❌ API Error: {e}")

1. Fetching Demographic Data from World Bank API...
✅ API Success! Retrieved 9044 rows.
Sütunlar: ['Country', 'Year', 'Total_Population', 'Active_Population_Total']


C:\Users\Caner\AppData\Local\Temp\ipykernel_7724\2081261847.py:8: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_api = wb.download(indicator=indicator_codes, country='all', start=1990, end=2023)


,Country,Year,Total_Population,Active_Population_Total
0,Afghanistan,1990,12045660.0,6032950.0
1,Afghanistan,1991,12238879.0,6109243.0
2,Afghanistan,1992,13278974.0,6602153.0
3,Afghanistan,1993,14943172.0,7397729.0
4,Afghanistan,1994,16250794.0,8005448.0


In [11]:
print("2. Loading and Cleaning Local CSV Data...")

# --- A. Load Olympics ---
if os.path.exists(OLYMPICS_PATH):
    df_olympics = pd.read_csv(OLYMPICS_PATH)
    # Clean Country Names
    df_olympics['Country'] = df_olympics['NOC'].str.replace(r'\s*\(.*\)', '', regex=True)
    df_olympics['Country'] = df_olympics['Country'].str.replace(r'\s*\[.*\]', '', regex=True)
    df_olympics['Country'] = df_olympics['Country'].str.strip()
    print(f"-> Olympics data loaded: {len(df_olympics)} rows.")
else:
    print(f"❌ Error: Olympics file not found at {OLYMPICS_PATH}")

# --- B. Load GDP ---
if os.path.exists(GDP_PATH):
    df_gdp = pd.read_csv(GDP_PATH, header=1)
    # Rename and Filter
    df_gdp.rename(columns={'Unnamed: 1': 'Country', 'Value': 'GDP_Per_Capita'}, inplace=True)
    df_gdp = df_gdp[df_gdp['Series'] == 'GDP per capita (US dollars)'].copy()
    # Numeric Conversion
    df_gdp['GDP_Per_Capita'] = pd.to_numeric(df_gdp['GDP_Per_Capita'].str.replace(',', ''), errors='coerce')
    print(f"-> GDP data loaded: {len(df_gdp)} rows.")
else:
    print(f"❌ Error: GDP file not found at {GDP_PATH}")

2. Loading and Cleaning Local CSV Data...
-> Olympics data loaded: 1436 rows.
-> GDP data loaded: 1694 rows.


In [13]:
print("3. Merging All Datasets...")

# Step 1: Merge GDP (CSV) and Population (API)
# Standard inner join on Country and Year
df_socio = pd.merge(
    df_gdp[['Country', 'Year', 'GDP_Per_Capita']], 
    df_api, 
    on=['Country', 'Year'], 
    how='inner'
)

# Step 2: Map Country Names (Olympics -> Standard)
country_map = {
    'United States': 'United States of America',
    'Great Britain': 'United Kingdom',
    'China': 'China',
    'Turkey': 'Türkiye',
    'Russia': 'Russian Federation',
    'South Korea': 'Republic of Korea',
    'Iran': 'Iran (Islamic Republic of)',
    'Syria': 'Syrian Arab Republic',
    'Vietnam': 'Viet Nam'
}
df_olympics['Country_Mapped'] = df_olympics['Country'].replace(country_map)

# Step 3: Merge Olympics with Socio-economic Data (Nearest Year)
df_olympics = df_olympics.sort_values('Year')
df_socio = df_socio.sort_values('Year')

df_final = pd.merge_asof(
    df_olympics,
    df_socio, 
    left_on='Year',
    right_on='Year',
    left_by='Country_Mapped',
    right_by='Country',
    direction='nearest',
    tolerance=5
)

# Filter out rows with missing core data
df_final = df_final.dropna(subset=['GDP_Per_Capita', 'Total_Population'])

# Save Result
output_file = 'processed_sports_gdp_population.csv'
df_final.to_csv('../data/processed/processed_sports_gdp_population.csv', index=False)

print("-" * 30)
print(f"🎉 SUCCESS! Final Dataset Saved: {output_file}")
print(f"Total Rows: {len(df_final)}")
display(df_final.head())

3. Merging All Datasets...
------------------------------
🎉 SUCCESS! Final Dataset Saved: processed_sports_gdp_population.csv
Total Rows: 608


,Year,Rank,NOC,Gold,Silver,Bronze,Total,Country_x,Country_Mapped,Country_y,GDP_Per_Capita,Total_Population,Active_Population_Total
695,1992,22,Norway,2,4,1,7,Norway,Norway,Norway,34875.0,4359184.0,2815261.0
696,1992,28,New Zealand,1,4,5,10,New Zealand,New Zealand,New Zealand,17189.0,3673400.0,2403555.0
697,1992,25,Brazil,2,1,0,3,Brazil,Brazil,Brazil,4781.0,161735073.0,101167654.0
698,1992,27,Sweden,1,7,4,12,Sweden,Sweden,Sweden,30520.0,8826939.0,5621611.0
699,1992,19,Poland,3,6,10,19,Poland,Poland,Poland,3705.0,38594998.0,25490654.0
